##Aufgabe 1:
a) Was sind Neuronale Netzwerke?
- rechnergestütze Modelle, die vom menschlichen Gehirn inspiriert sind
- bestehen aus mehreren Schichten von input layer bis output layer
- jede Schicht besteht aus Neuronen, die Rechenoperationen durchführen um Muster zu erkennen
- lernen indem sie die internen Gewichte und Biases anpassen
- durch nichtlineare Aktivierungsfunktionen können NNs dabei auch sehr komplexe Funktionen annähern

b) Was ist die Chain Rule?
- Die chain rule ist ein mathematischer Trick, bei dem man erkennt was eine Veränderung am Anfang (bspw. weight) auf das Ergebnis am Ende (bspw. den Fehler) hat.

c) Was versteht man unter dem XOR-Problem? Welche Folgen hatte es und welche Lösungen wurden gefunden?
- Ein lineares neuronales kann XOR nicht lernen da es in einem Koordinatensystem nicht mit einer geraden Linie getrennt werden kann
- mit mindestens einem hidden layer und einer nichtlinearen Aktivierungsfunktion kann ein neural network XOR lernen

d)Beschreibe zwei beliebige Aktivierungsfunktionen.
ReLU
- 0 wenn z < 0
- z wenn z > 0
- sehr effizient
- funktioniert gut in tiefen Netzen

Sigmoid
- Werte zwischen 0 und 1
- kleine und große eingaben werden abgeflacht

e) Wie funktioniert Backpropagation?
1. Forward Pass:
- Alle Eingaben laufen durchs Netzwerk
- Output (Vorhersage) bspw. 0.87
2. Loss
- Wert zwischen Vorhersage und echtem Wert messen
- Das ist der Fehler
3. Backward Pass (Chain Rule)
- Wie hat jedes Gewicht zum Loss beigetragen
- Dafür verwenden wir die Chain Rule
- Das passiert für jede Schicht rückwärts, von der letzten bis zur ersten
4. Gradient Descent
- Das Gewicht wird so angepasst, um den Fehler beim nächsten Mal kleiner zu machen


## Aufgabe 2
Skizziere ein simples neuronales Netz, welches die folgenden logischen Ausdrücke repräsentiert.
a) (a OR b) AND (c OR d)
b) NOT(a OR b) AND (c OR d)
c) (a OR b) XOR (c OR d)

## Aufgabe 3

In dieser Übung wollen wir ein neuronales Netzwerk zur Word-Prediktion erstellen. Dafür nutzen wir PyTorch. Installieren Sie das Package via `pip install torch`.

Hierfür soll das Model, ähnlich wie bei N-Grams, anhand des letzten Tokens (in diesem Fall: das letzte Wort) bestimmen, welches das nächste Token (Wort) ist. Bonus: Begrenze die Kontextlänge nicht nur auf das letzte Wort und implementiere eine variable Kontextlänge, zum Beispiel die letzten 3 Wörter.

Präparieren Sie einen Datensatz aus den Gutenberg Corpora und nutzen Sie PyTorch, um Ihr Model zu definieren und zu trainieren.

Es stehen zwei Notebooks zur Verfügung, wobei `uebung02.ipynb` die Struktur ohne weitere Hilfestellungen vorschlägt und `uebung02_assissted.ipynb` die Definition von Model und Training bereitstellt, sodass Sie sich nur auf die Vorbereitung der Trainingsdaten und die Prediction Schritte konzentrieren können. Das Ziel ist jedoch in beiden Fällen dasselbe.

Nutzen Sie zur Definition des NN die Ihnen von der Vorlesung bekannten Elemente: Lineare Layer, eine beliebige Aktivierungsfunktion und Backpropagation im Training.

Lassen Sie sich mit dem fertigen Model einige Beispiele für Folgewörter generieren.

Für Hilfestellungen, hier ein paar Quellen:
Lineare Layer: https://docs.kanaries.net/topics/Python/nn-linear
Textgeneration mit RNN: https://abhijoysarkar.blog/2024/05/10/building-a-mini-language-model-with-pytorch-tutorial-walkthrough/ (Achtung: RNN sind noch nicht in der Vorlesung drangekommen - das Model muss entsprechend angepasst werden)


In [8]:
import nltk
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from nltk.corpus import gutenberg
from nltk.tokenize import word_tokenize, sent_tokenize
from collections import Counter
import random
import numpy as np

nltk.download('gutenberg')
nltk.download('punkt')
nltk.download('punkt_tab')
raw_text = gutenberg.raw('austen-emma.txt')

sentences = sent_tokenize(raw_text)

[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [11]:
context_size = 3

allWords = set()
document = [" ".join(word_tokenize(sentence)) for sentence in sentences]
for line in document:
  words = ["<start>"] + line.split() + ["<end>"]
  allWords.update(words)

wtoi = {w:i for i, w in enumerate(sorted(allWords))}
itow = {i:w for w, i in wtoi.items()}
vocab_size = len(wtoi)

In [12]:
xs = []
ys = []

for line in document:
  words = ["<start>"] + line.split() + ["<end>"]
  for i in range(len(words) - context_size):
    context = words[i:i+context_size]
    target = words[i+context_size]
    xs.append([wtoi[w] for w in context])
    ys.append(wtoi[target])

inputs_tensor = torch.tensor(xs, dtype=torch.long)
outputs_tensor = torch.tensor(ys, dtype=torch.long)

In [13]:
# Data Management beim Training
#PyTorch’s Dataset class: allows us to define how to access our data in a format suitable for feeding it into our model
# DataLoader handles batching of data, shuffling, and parallel processing.

# Combine into dataset
class WordDataset(Dataset):
    def __init__(self, xs, ys):
        self.xs = xs
        self.ys = ys

    def __len__(self):
        return len(self.xs)

    def __getitem__(self, index):
        return self.xs[index], self.ys[index]

dataset = WordDataset(inputs_tensor, outputs_tensor)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

In [14]:
class WordPredictor(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, context_size):
        # Definiert die Struktur vom Predictor
        super().__init__()
        # Embedding
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        # Layer 1: Linear
        self.fc1 = nn.Linear(embed_dim * context_size, hidden_dim)
        # Activation Function: ReLu
        self.relu = nn.ReLU()
        # Layer 2: Linear
        self.fc2 = nn.Linear(hidden_dim, vocab_size)

    def forward(self, x):
        """Methode wird immer aufgerufen, wenn Predictions gemacht werden (Training und Inferenz)"""

        # Step 1: Wort-Indexe zu Embeddings konvertieren und glätten -> Kontext Wörter in einen Vektor, der vom linearen Layer verwendet werden kann
        x = self.embedding(x).view(x.size(0), -1)

        # Step 2: Durch das 1. Layer -> Lineare Transformation, um Inputs in das versteckte Layer zu projizieren
        x = self.fc1(x)

        # Step 3: ReLU activation function
        x = self.relu(x)

        # Step 4: 2. fully connected layer -> Wieder auf die Vokabelgröße projizieren
        return self.fc2(x)

model = WordPredictor(vocab_size, embed_dim=64, hidden_dim=128, context_size=context_size)

In [15]:
# Loss function -> Misst wie gut die Model Prediction dem richtigen Output entspricht
criterion = nn.CrossEntropyLoss()
# Optimizer -> Updatet die Gewichte und Parameter um Loss ^ zu minimieren (in dem Fall Gradient Descent)
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [16]:
no_of_epochs = 10

# Durchgehen durch die Trainingsepochen
for epoch in range(no_of_epochs):
    # Loss für Dokumentation
    total_loss = 0
    for batch_x, batch_y in dataloader:
        optimizer.zero_grad()
        logits = model(batch_x)

        # Loss berechnen
        loss = criterion(logits, batch_y)
        # Backpropagation
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {total_loss:.4f}")

Epoch 1, Loss: 30808.8581
Epoch 2, Loss: 26802.3455
Epoch 3, Loss: 25075.0818
Epoch 4, Loss: 23941.0579
Epoch 5, Loss: 23069.5812
Epoch 6, Loss: 22341.0500
Epoch 7, Loss: 21732.0623
Epoch 8, Loss: 21182.2555
Epoch 9, Loss: 20698.2884
Epoch 10, Loss: 20255.1737


In [19]:
def generate_sentence(model, start_words, length=10):
    model.eval()
    words = start_words[:]
    for _ in range(length):
        context = words[-context_size:]
        # Padding falls zu kurz
        if len(context) < context_size:
            context = ["<start>"] * (context_size - len(context)) + context
        context_ids = [wtoi.get(w, wtoi["<start>"]) for w in context]
        x = torch.tensor([context_ids], dtype=torch.long)
        with torch.no_grad():
            logits = model(x)
            next_word_idx = torch.argmax(logits, dim=1).item()
            next_word = itow[next_word_idx]
        words.append(next_word)
        if next_word == "<end>":
            break
    return " ".join(words)

# Beispiel:
start = ["she", "was", "very"]
print("Generiert:", generate_sentence(model, start))


Generiert: she was very much pleased with the first , and the same party
